In [ ]:
from multiinstance.data.realData import buildDataset as buildReal
from multiinstance.utils import *

from multiinstance.ward_clustering import WardClustering

import matplotlib.pyplot as plt

from glob import glob
from tqdm.notebook import tqdm

import numpy as np

In [ ]:
for f in tqdm(glob("/ssdata/ClassPriorEstimationPrivate/data/rawDatasets/*.mat")):
    ds = buildReal(f,16,nPDistr=lambda: 1 + np.random.poisson(25),
                   nUDistr=lambda: 1 + np.random.poisson(75))

    ds = addTransformScores(ds)


    ds.alphaHats,ds.curves = getBagAlphaHats(ds,numbootstraps=100)

    ward = WardClustering(ds,randomPairing=True,numbootstraps=ds.alphaHats.shape[1])
    ward.cluster()

    meanvars = []
    for level in range(ward.alphaHatMat.shape[0]):
        c_labels = np.unique(ward.clusterAssignment[level]).astype(int)
        meanvars.append(ward.alphaHatMat[level,c_labels].var(1).mean())

    plt.plot(meanvars)
    plt.show()

In [ ]:
ds= buildReal("/data/dzeiberg/ClassPriorEstimation/rawDatasets/abalone.mat", 4)
ds = addTransformScores(ds)
ds.alphaHats, ds.curves = getBagAlphaHats(ds,numbootstraps=10)
ward = WardClustering(ds,randomPairing=True,numbootstraps=ds.alphaHats.shape[1])
ward.cluster()

In [ ]:
for level in range(ward.clusterAssignment.shape[0]):
    cs = np.unique(ward.clusterAssignment[level]).astype(int)
    aes = []
    for c in cs:
        bags = np.where(ward.clusterAssignment[level] == c)[0]
        alpha = np.dot(ds.trueAlphas.flatten()[bags],ds.numU[bags]) / ds.numU[bags].sum()
        alphaHat= ward.alphaHatMat[level][c]
        ae = np.abs(alpha - alphaHat)
        aes.append(ae)
    print("level: {}    MAE: {:.3f}".format(level, np.mean(aes)))